# Automatic values

Auto generationg associated values. Behind the scenes, uses a method in th the Enum class, the `_generate_next_value_` method

By default, implementation results in sequential integer numbers 

### `_generate_next_value_(name, start, count, last_values)`

`name` - is the anme of the member

`start` - only actually used in functional creation (not covered in the course, not used that often)

`count` - the numbers of members already created (careful with aliasses)

`last_values` a list of preceding values

returns:

`value` - to be assigne to member

#### Overriding 

The default implementation gernerates sequential integer numbers. Be careful mixnig auto() and our own values. 

__Safer not to do it!__

In [2]:
import enum

In [3]:
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [4]:
for member in State:
    print(member.name, member.value)

WAITING 1
STARTED 2
FINISHED 3


In [5]:
class State(enum.Enum):
    WAITING = 100
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [6]:
list(State)

[<State.WAITING: 100>, <State.STARTED: 101>, <State.FINISHED: 102>]

Do not rely on the complementation!! That isn't in the documentation and could change anytime

In [7]:
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = 1
    FINISHED = enum.auto()

In [8]:
list(State)

[<State.WAITING: 1>, <State.FINISHED: 2>]

In [9]:
@enum.unique
class State(enum.Enum):
    WAITING = enum.auto()
    STARTED = 1
    FINISHED = enum.auto()

ValueError: duplicate values found in <enum 'State'>: STARTED -> WAITING

In [13]:
State.__members__

mappingproxy({'WAITING': <State.WAITING: 100>,
              'STARTED': <State.STARTED: 101>,
              'FINISHED': <State.FINISHED: 102>})

In [14]:
hasattr(State, '_generate_next_value_')

True

If has a default implementation we can override it

In [20]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_value):
        print(f"name={name}, start={start}, count={count}, last_value={last_value}")
        return 100
    
    a = enum.auto()
    b = enum.auto()
    c = enum.auto()


name=a, start=1, count=0, last_value=[]
name=b, start=1, count=1, last_value=[100]
name=c, start=1, count=2, last_value=[100, 100]


In [22]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_value):
        print(f"name={name}, start={start}, count={count}, last_value={last_value}")
        return 100
    
    a = enum.auto()
    b = enum.auto()
    d = 200
    c = enum.auto()


name=a, start=1, count=0, last_value=[]
name=b, start=1, count=1, last_value=[100]
name=c, start=1, count=3, last_value=[100, 100, 200]


Notice that the last value list is a __element ordered__ list

In [14]:
import random

random.seed(0)

class State(enum.Enum):
    def _generate_next_value(name, start, count, last_values):
        while True:
            new_value = random.randint(1, 100)
            if new_value not in last_values:
                return new_value
            
    a = enum.auto()
    b = enum.auto()
    c = enum.auto()
    d = enum.auto()

In [15]:
for item in State:
    print(item.name, item.value)

a 1
b 2
c 3
d 4


In [ ]:
class State(enum.Enum):
    READY = 'ready'
    WAITING = 'waiting'
    

In order to not have to repeat the lowercase creation of states, we can override the `_generate_next_value_` mathod 

In [16]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.title()
    
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [17]:
for item in State:
    print(item.name, item.value)

WAITING Waiting
STARTED Started
FINISHED Finished


This autogenerates a string value

To make it even more reusable, we can mix that with subclassing:

In [18]:
class NameAsString(enum.Enum):
    def _generate_next_value_(name, start, count, last_value):
        return name.lower()

In [21]:
class State(NameAsString):
    WAITING = enum.auto()
    STARTED = enum.auto()
    FINISHED = enum.auto()

In [22]:
list(State)

[<State.WAITING: 'waiting'>,
 <State.STARTED: 'started'>,
 <State.FINISHED: 'finished'>]

In [27]:
class State(enum.Enum):
    """Please do not use member values - these are maningles and subject to change"""
    Waiting = 1 
    Running = 2 
    Finished = 3 

In [30]:
State.Waiting, State['Waiting']

(<State.Waiting: 1>, <State.Waiting: 1>)

In [31]:
State(1) # but the user still can pushes using ghe value

<State.Waiting: 1>

In [32]:
class State(enum.Enum):
    Waiting = object()
    Running = object()
    Finished = object()

This way, the values are meaningless. We're only intrusted in the keys, in the names, so we're avoiding using the values

In [34]:
State.Waiting, State['Finished']

(<State.Waiting: <object object at 0x000001E97F4288C0>>,
 <State.Finished: <object object at 0x000001E97F4286B0>>)

In [35]:
class Aliased(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(f"count={count}")
        if count % 2 == 1: # odd number 
            # make this member an alies of the previous one
            return last_values[-1]
        else:
            return last_values[-1] + 1 #never use auto for the first element
        
    GREEN = 1 
    GREEN_ALIAS = 1 
    RED = 10 
    CRIMSON = enum.auto()
    BLUE = enum.auto()
    AQUA = enum.auto()

count=3
count=4
count=5


In [36]:
list(Aliased)

[<Aliased.GREEN: 1>, <Aliased.RED: 10>, <Aliased.BLUE: 11>]

In [37]:
Aliased.__members__

mappingproxy({'GREEN': <Aliased.GREEN: 1>,
              'GREEN_ALIAS': <Aliased.GREEN: 1>,
              'RED': <Aliased.RED: 10>,
              'CRIMSON': <Aliased.RED: 10>,
              'BLUE': <Aliased.BLUE: 11>,
              'AQUA': <Aliased.BLUE: 11>})